# 🧈 Training MOCOv2 on CIFAR10 🔥 With pytorch lightning 🧈

In [1]:
import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl
import lightly

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import loggers as pl_loggers

SEED = 1

/home/shatz/miniconda3/envs/morebetter3/lib/python3.9/site-packages/lightly/api/version_checking.py:57: Warning: You are using lightly version 1.1.18. There is a newer version of the package available. For compatability reasons, please upgrade your current version: pip install lightly==1.1.20
  warnings.warn(Warning(warning))


# ⬇️ Build Dataset

In [2]:
# DATA hyperparams
num_workers = 6
moco_batch_size = 512
classifier_train_batch_size = 512
classifier_test_batch_size = 512

In [3]:
# The dataset structure should be like this:
# cifar10/train/
#  L airplane/
#    L 10008_airplane.png
#    L ...
#  L automobile/
#  L bird/
#  L cat/
#  L deer/
#  L dog/
#  L frog/
#  L horse/
#  L ship/
#  L truck/
path_to_train = './data/cifar10_lightly/train/'
path_to_test = './data/cifar10_lightly/test/'

### Augmentations

In [4]:
################### Classifier Augmentations ###################
# Augmentations typically used to train on cifar-10
train_classifier_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=lightly.data.collate.imagenet_normalize['mean'],
        std=lightly.data.collate.imagenet_normalize['std'],
    )
])

# No additional augmentations for the test set
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32, 32)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=lightly.data.collate.imagenet_normalize['mean'],
        std=lightly.data.collate.imagenet_normalize['std'],
    )
])

################### MOCO Augmentations ###################
# MoCo v2 uses SimCLR augmentations, additionally, disable blur
collate_fn = lightly.data.SimCLRCollateFunction(
    input_size=32,
    gaussian_blur=0.,
)

### Datasets

In [5]:
################### Classifier Datasets ###################
#Since we also train a linear classifier on the pre-trained moco model we
# reuse the test augmentations here (MoCo augmentations are very strong and
# usually reduce accuracy of models which are not used for contrastive learning.
# Our linear layer will be trained using cross entropy loss and labels provided
# by the dataset. Therefore we chose light augmentations.)
dataset_train_classifier = lightly.data.LightlyDataset(
    input_dir=path_to_train,
    transform=train_classifier_transforms
)

dataset_test = lightly.data.LightlyDataset(
    input_dir=path_to_test,
    transform=test_transforms
)

################### MOCO Dataset ###################
# We use the moco augmentations for training moco
dataset_train_moco = lightly.data.LightlyDataset(
    input_dir=path_to_train
)

### Dataloaders

In [6]:
################### Classifier Dataloaders ###################
dataloader_train_classifier = torch.utils.data.DataLoader(
    dataset_train_classifier,
    batch_size=classifier_train_batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=classifier_test_batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

################### MOCO Dataloader ###################
dataloader_train_moco = torch.utils.data.DataLoader(
    dataset_train_moco,
    batch_size=moco_batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers
)

# 🧠 Load Model

In [7]:
# MODEL hyperparams
memory_bank_size = 4096
moco_max_epochs = 9005
downstream_max_epochs = 60
downstream_test_every = 300

In [8]:
import moco_model

In [9]:
model = moco_model.MocoModel(memory_bank_size, moco_max_epochs, 
                             downstream_max_epochs, dataloader_train_classifier, dataloader_test,
                            downstream_test_every=downstream_test_every)
#                             last_epoch=2989)

I will continue training from 900 epoch model

In [10]:
# # # WHEN LOADING A SAVED MODEL DO IT LIKE DIS
# model.load_from_checkpoint('./saved_models/resnet_moco/epoch=2989-train_loss_ssl=1.37.ckpt',
#                           memory_bank_size=memory_bank_size, moco_max_epochs=moco_max_epochs,
#                           downstream_max_epochs=downstream_max_epochs, dataloader_train_classifier=dataloader_train_classifier,
#                            dataloader_test=dataloader_test,
#                            downstream_test_every=downstream_test_every)
# #                           last_epoch=2989);
# # checkpoint_callback = ModelCheckpoint(dirpath="./saved_models/resnet_moco/epoch=897-train_loss_ssl=1.67.ckpt")

In [11]:
# you can also define a checkpoint callback to save best model like keras.
checkpoint_callback = ModelCheckpoint(
    dirpath='./saved_models/resnet_moco',
    filename='{epoch}-{train_loss_ssl:.2f}',
    save_top_k=5,
    verbose=True,
    monitor='train_loss_ssl',
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

/home/shatz/miniconda3/envs/morebetter3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory ./saved_models/resnet_moco exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


# 🏋️‍♂️ Train

In [12]:
# use a GPU if available
gpus = 1 if torch.cuda.is_available() else 0
print(f'Using gpu: {bool(gpus)}')
if(gpus == 0): print('--- NOT USING GPUS THIS TAKE LONG TIME ---')

# set up tensorboard logger
tb_logger = pl_loggers.TensorBoardLogger(save_dir='./lightning_logs/', name=f'TESTmoco_{moco_max_epochs}eps')

Using gpu: True


In [13]:
# ### WHEN TRAINING FROM CHECKPOINT ###

# trainer = pl.Trainer(
#     resume_from_checkpoint="./saved_models/resnet_moco/epoch=2989-train_loss_ssl=1.37.ckpt",
#     max_epochs=moco_max_epochs,
#     gpus=gpus,
#     callbacks=[checkpoint_callback],
#     logger=tb_logger)
# trainer.fit(model, dataloader_train_moco)

In [14]:
## WHEN TRAINING FROM SCRATCH (EPOCH 0) ###

trainer = pl.Trainer(max_epochs=moco_max_epochs, gpus=gpus, callbacks=[checkpoint_callback, lr_monitor], logger=tb_logger)
trainer.fit(
    model,
    dataloader_train_moco
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | resnet_moco | MoCo       | 23.0 M
1 | criterion   | NTXentLoss | 0     
-------------------------------------------
11.5 M    Trainable params
11.5 M    Non-trainable params
23.0 M    Total params
91.977    Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

Epoch 0, global step 96: train_loss_ssl reached 6.96062 (best 6.96062), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=0-train_loss_ssl=6.96.ckpt" as top 5
Epoch 1, global step 193: train_loss_ssl reached 7.10845 (best 6.96062), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=1-train_loss_ssl=7.11.ckpt" as top 5
Epoch 2, global step 290: train_loss_ssl reached 7.05138 (best 6.96062), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=2-train_loss_ssl=7.05.ckpt" as top 5
Epoch 3, global step 387: train_loss_ssl reached 6.93874 (best 6.93874), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=3-train_loss_ssl=6.94.ckpt" as top 5
Epoch 4, global step 484: train_loss_ssl reached 6.62397 (best 6.62397), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=4-train_loss_ssl=6.62.ckpt" as top 5
Epoch 5, global step 581: train_loss_ssl 

Epoch 44, global step 4364: train_loss_ssl reached 3.54169 (best 3.54169), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=44-train_loss_ssl=3.54.ckpt" as top 5
Epoch 45, global step 4461: train_loss_ssl reached 3.53394 (best 3.53394), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=45-train_loss_ssl=3.53.ckpt" as top 5
Epoch 46, global step 4558: train_loss_ssl reached 3.60386 (best 3.53394), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=46-train_loss_ssl=3.60.ckpt" as top 5
Epoch 47, global step 4655: train_loss_ssl reached 3.60247 (best 3.53394), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=47-train_loss_ssl=3.60.ckpt" as top 5
Epoch 48, global step 4752: train_loss_ssl reached 3.59405 (best 3.53394), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=48-train_loss_ssl=3.59.ckpt" as top 5
Epoch 49, global step 484

Epoch 100, global step 9796: train_loss_ssl reached 2.96444 (best 2.86490), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=100-train_loss_ssl=2.96.ckpt" as top 5
Epoch 101, global step 9893: train_loss_ssl was not in top 5
Epoch 102, global step 9990: train_loss_ssl reached 2.95429 (best 2.86490), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=102-train_loss_ssl=2.95.ckpt" as top 5
Epoch 103, global step 10087: train_loss_ssl reached 2.91635 (best 2.86490), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=103-train_loss_ssl=2.92.ckpt" as top 5
Epoch 104, global step 10184: train_loss_ssl reached 2.93891 (best 2.86490), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=104-train_loss_ssl=2.94.ckpt" as top 5
Epoch 105, global step 10281: train_loss_ssl was not in top 5
Epoch 106, global step 10378: train_loss_ssl was not in top 5
Epoch 107, global step 10

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 9000, global step 873096: train_loss_ssl was not in top 5


[1.0914463, 0.7046748, 0.5597423, 0.483434, 0.43608052, 0.402711, 0.37839693, 0.36016092, 0.34510458, 0.33263665, 0.32219622, 0.31369197, 0.30627978, 0.29951453, 0.29381943, 0.2884774, 0.28368464, 0.27943724, 0.2755902, 0.27187037, 0.26840302, 0.2652933, 0.26232028, 0.25962734, 0.25696343, 0.2545145, 0.25219604, 0.24999496, 0.24791953, 0.24593328, 0.24412307, 0.24228661, 0.24062507, 0.23897389, 0.23739769, 0.23592857, 0.23444444, 0.23306134, 0.23172319, 0.23046157, 0.22925471, 0.22808005, 0.2268935, 0.22576961, 0.22474274, 0.22368304, 0.22270638, 0.22174245, 0.22081, 0.21988136, 0.21898659, 0.21815291, 0.21737158, 0.21656826, 0.21584915, 0.21512158, 0.21440914, 0.21374178, 0.21306185, 0.2124432]
[0.62263036, 0.7130346, 0.75434357, 0.779239, 0.79603523, 0.8084562, 0.81812966, 0.82583565, 0.8322191, 0.83757097, 0.8421413, 0.84608686, 0.8495537, 0.85257983, 0.855272, 0.8576955, 0.8598818, 0.86186415, 0.863671, 0.86533767, 0.8668742, 0.86829793, 0.8696145, 0.8708404, 0.87198657, 0.8730599,

Epoch 9001, global step 873193: train_loss_ssl was not in top 5
Epoch 9002, global step 873290: train_loss_ssl was not in top 5
Epoch 9003, global step 873387: train_loss_ssl was not in top 5
Epoch 9004, global step 873484: train_loss_ssl was not in top 5
